In [1]:
from phi.torch.flow import *
import matplotlib.pyplot as plt

from src.eigenfluid import Eigenfluid
from src.shapes import *

# Helper functions for visualization
from src.visu import *

import time

In [2]:
# Number of basis fields
N = 4**2
# Domain sizes
DOMAIN = Box(x=math.PI, y=math.PI)
# NOT for sampling points, but for reconstructing the velocity grid
SAMPLING_SIZE = 32

BATCH_SIZE = 200
VISU_NR = 5

start_time = time.time()
eigenfluid = Eigenfluid(N, DOMAIN, SAMPLING_SIZE, init_w='zero')
end_time = time.time()
print("Eigenfluid initialized in {} seconds".format(end_time-start_time))

Eigenfluid initialized in 7.40953803062439 seconds


# Data

In [3]:
# Get O overlapping, and U non-necessarily overlapping ('unique') sample points
O = 12
U = 5
N_S = O+U

def get_points_for_shapes(shape_0, shape_target):
    sampler_union = ShapeSampler(shape_0, shape_target, N=O, h1=2, h2=7)
    sampler_0 = ShapeSampler(shape_0, N=U, h1=3, h2=11)
    sampler_target = ShapeSampler(shape_target, N=U, h1=3, h2=11)
    # Concatenate both the union and non-union points
    p_0 = math.concat((sampler_union.p, sampler_0.p), instance('i'))
    p_0 = shape_0.create_points(p_0)
    p_t = math.concat((sampler_union.p, sampler_target.p), instance('i'))
    p_t = shape_target.create_points(p_t)
    
    return (p_0, p_t)

p_0_list = []
p_t_list = []
for i in range(BATCH_SIZE):
    # Circle / Square / Triangle
    Shape_0 = [Circle, Square, Triangle][np.random.randint(3)]
    Shape_target = [Circle, Square, Triangle][np.random.randint(3)]
    # Random positions [0,2] x [0,2]
    shape_0 = Shape_0(pos=(np.random.rand()*2, np.random.rand()*2))
    shape_target = Shape_target(pos=(np.random.rand()*2, np.random.rand()*2))
    
    p_0, p_t = get_points_for_shapes(shape_0, shape_target)
    p_0_list.append(p_0)
    p_t_list.append(p_t)

    
p_0 = math.stack(p_0_list, batch(batch=BATCH_SIZE))
p_t = math.stack(p_t_list, batch(batch=BATCH_SIZE))

# Generate transition data
STEPS = 16 #p_0 is at time step 1
p_trajectory = math.tensor([p_0+(p_t - p_0)/STEPS*t for t in range(STEPS)], batch('time') & p_0.shape)

plot({
    "Blue: start shape\nRed: target shape\nGreen: Linear Trajectory": vis.overlay(
        PointCloud(p_0.batch[:VISU_NR], bounds=DOMAIN),
        PointCloud(p_t.batch[:VISU_NR], bounds=DOMAIN, color="red"),
        PointCloud(p_trajectory.batch[:VISU_NR], bounds=DOMAIN, color="green"),
     )}, 
     animate='time',
     size=(12,4))

<Figure size 640x480 with 0 Axes>

# Loss Function

In [31]:
STEPS = 16 # Already defined during data generation
DT    = 0.2
VISCOSITY = 0.0

def to_model(p_0, p_t, w):
    return math.stack([
        p_0['x'], p_0['y'],
        p_t['x'], p_t['y'],
        math.pad(math.rename_dims(w, 'k', instance('i')), {'i': (0,N_S-N)}, extrapolation.ZERO)
    ], channel('channels'))

def to_model_flat(p_0, p_t, w):
    return math.concat([
        p_0['x'], p_0['y'],
        p_t['x'], p_t['y'],
        math.rename_dims(w, 'k', instance('i'))
    ], instance('i'))

# Get the 4th channel, and reshape it to match the dimensions of w (=> instance('k'))
def from_model(net_output):
    return math.rename_dims(tensor(
        net_output,
        batch('batch') & channel(channels) & channel('vector')
    ).channels[4], 'vector', instance('k'))

def from_model_flat(net_output):
    return math.rename_dims(tensor(
        net_output,
        batch('batch') & channel('channels') & channel('vector')
    ).channels[0], 'vector', instance('k'))


@math.jit_compile
def loss_function(p_0, p_t, w):
    loss = 0
    p_est = p_0.time[0]
    p_est_traj = [p_est]
    for t in range(STEPS-1):
        # Feed data into NN to get CFE prediction
        model_input = to_model_flat(p_est, p_t.time[t], w)
        model_output = net(model_input.native('batch,channels,i'))
        f_pred = from_model_flat(model_output)
        #print("t={}, f_pred={}".format(t, f_pred))
        w += f_pred
        # Step simulation
        w = eigenfluid.step_w_euler(w, DT, viscosity=VISCOSITY)
        # Get velocities at p_0
        v_phi = eigenfluid.phi_template(w, eigenfluid.N, eigenfluid.basis_fields)
        p_velocities = v_phi(p_est)
        # Midpoint
        p_velocities = v_phi(p_est + p_velocities*DT/2)
        # Euler step for points
        p_est += p_velocities * DT
        p_est_traj.append(p_est)
        loss += math.l2_loss(p_est - p_t.time[t])
    return loss, p_est_traj

# Network and Training

In [32]:
# Input: [p_current, p_target, current_w]
# -> Input dimension = [O+U, O+U, N]
# Output = [N] force coefficients for current time step
#net = dense_net(2*2*(O+U)+N, N, [256, 128, 64, 32], batch_norm=True)
net = dense_net(N_S*2*2+N, N, [256, 128, 64, 32])
optimizer = adam(net)

print(net)
print("Number of parameters: {}".format(parameter_count(net)))
#print()
#print("Optimizer:{}".format(optimizer))

DenseNet(
  (linear0): Linear(in_features=84, out_features=256, bias=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=16, bias=True)
)
Number of parameters: 65520


### Visualize Untrained CFE NN (first 5)

In [33]:
## Test the loss function and the NN
loss, cfe_traj = loss_function(
                    p_trajectory.time[0:STEPS-1].batch[:VISU_NR],
                    p_trajectory.time[1:STEPS].batch[:VISU_NR],
                    math.stack([eigenfluid.w]*VISU_NR, batch('batch'))
                )
print("Loss = {}".format(loss))
cfe_traj = math.stack(cfe_traj, batch('time'))
plot(PointCloud(cfe_traj, bounds=DOMAIN), animate='time')

Loss = (140.955, 109.799, 93.084, 128.127, 130.571) along batchᵇ


<Figure size 640x480 with 0 Axes>

## Train

In [34]:
for epoch in range(1000):
    w_optim = math.stack([eigenfluid.w]*BATCH_SIZE, batch('batch'))
    loss, p_est_traj = update_weights(net, optimizer, loss_function, 
                   p_trajectory.time[0:STEPS-1], 
                   p_trajectory.time[1:STEPS], 
                   w_optim)
    if epoch%10 == 0 or epoch < 5:
        print("Optimization step {}, loss: {}".format(epoch, loss))

D:\Barney\conda-install\envs\eigenfluid-control\lib\site-packages\phi\torch\_torch_backend.py:807: RuntimeWarning: PyTorch does not support nested tracing. The inner JIT of native(loss_function) will be ignored.
  warnings.warn(f"PyTorch does not support nested tracing. The inner JIT of {self.f.__name__} will be ignored.", RuntimeWarning)


Optimization step 0, loss: (batchᵇ=200) 113.839 ± 43.925 (4e+01...3e+02)
Optimization step 1, loss: (batchᵇ=200) 104.845 ± 42.036 (3e+01...3e+02)
Optimization step 2, loss: (batchᵇ=200) 96.868 ± 41.497 (3e+01...2e+02)
Optimization step 3, loss: (batchᵇ=200) 94.322 ± 39.931 (3e+01...2e+02)
Optimization step 4, loss: (batchᵇ=200) 93.122 ± 39.802 (3e+01...2e+02)
Optimization step 10, loss: (batchᵇ=200) 81.717 ± 41.572 (2e+01...2e+02)
Optimization step 20, loss: (batchᵇ=200) 58.262 ± 36.008 (8e+00...2e+02)
Optimization step 30, loss: (batchᵇ=200) 38.220 ± 23.876 (3e+00...1e+02)
Optimization step 40, loss: (batchᵇ=200) 26.809 ± 18.317 (4e+00...1e+02)
Optimization step 50, loss: (batchᵇ=200) 19.368 ± 13.263 (2e+00...7e+01)
Optimization step 60, loss: (batchᵇ=200) 13.376 ± 8.841 (2e+00...5e+01)
Optimization step 70, loss: (batchᵇ=200) 11.437 ± 7.904 (1e+00...5e+01)
Optimization step 80, loss: (batchᵇ=200) 9.544 ± 7.127 (1e+00...5e+01)
Optimization step 90, loss: (batchᵇ=200) 8.257 ± 6.454 (1e

KeyboardInterrupt: 

## Visualize Results (first 5)

In [35]:
loss, cfe_traj = loss_function(
                    p_trajectory.time[0:STEPS-1].batch[15:15+VISU_NR],
                    p_trajectory.time[1:STEPS].batch[15:15+VISU_NR],
                    math.stack([eigenfluid.w]*VISU_NR, batch('batch'))
                )
print("Loss = {}".format(loss))
cfe_traj = math.stack(cfe_traj, batch('time'))
plot(
    vis.overlay(
        PointCloud(cfe_traj, bounds=DOMAIN),
        PointCloud(p_trajectory.time[STEPS-1].batch[15:15+VISU_NR], bounds=DOMAIN, color='red')
    ),
    animate='time'
)

Loss = (2.071, 1.437, 2.956, 0.671, 0.424) along batchᵇ


<Figure size 640x480 with 0 Axes>

# Test Generalization of NN on Unseen Data

## Generate Test Data

In [36]:
TEST_SIZE = 10

# Generate test data
p_0_test_list = []
p_t_test_list = []
for i in range(TEST_SIZE):
    # Circle / Square / Triangle
    Shape_0 = [Circle, Square, Triangle][np.random.randint(3)]
    Shape_target = [Circle, Square, Triangle][np.random.randint(3)]
    # Random positions [0,2] x [0,2]
    shape_0 = Shape_0(pos=(np.random.rand()*2, np.random.rand()*2))
    shape_target = Shape_target(pos=(np.random.rand()*2, np.random.rand()*2))
    
    p_0, p_t = get_points_for_shapes(shape_0, shape_target)
    p_0_test_list.append(p_0)
    p_t_test_list.append(p_t)

    
p_0_test = math.stack(p_0_test_list, batch(batch=TEST_SIZE))
p_t_test = math.stack(p_t_test_list, batch(batch=TEST_SIZE))

# Generate transition data
p_test_trajectory = math.tensor([p_0_test+(p_t_test - p_0_test)/STEPS*t for t in range(STEPS)], batch('time') & p_0_test.shape)

## Run Tests

In [37]:
loss, test_traj = loss_function(
                    p_test_trajectory.time[0:STEPS-1],
                    p_test_trajectory.time[1:STEPS],
                    math.stack([eigenfluid.w]*TEST_SIZE, batch('batch'))
                )
print("Loss = {}".format(loss))
test_traj = math.stack(test_traj, batch('time'))
plot(
    vis.overlay(
        PointCloud(test_traj.batch[:VISU_NR], bounds=DOMAIN),
        PointCloud(p_t_test.batch[:VISU_NR], bounds=DOMAIN, color="red"),
    ),
    animate='time'
)

D:\Barney\conda-install\envs\eigenfluid-control\lib\site-packages\phi\torch\_torch_backend.py:807: RuntimeWarning: PyTorch does not support nested tracing. The inner JIT of native(loss_function) will be ignored.
  warnings.warn(f"PyTorch does not support nested tracing. The inner JIT of {self.f.__name__} will be ignored.", RuntimeWarning)


Loss = (batchᵇ=10) 3.378 ± 3.652 (6e-01...1e+01)


<Figure size 640x480 with 0 Axes>